In [ ]:
# AWS RDS 연결
import mysql.connector

aws_DB = mysql.connector.connect(
    host = "database-1.ciifx43v3wkq.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "qaz51133"
)

In [ ]:
# aws RDS 연결
aws_DB = mysql.connector.connect(
    host = "database-1.ciifx43v3wkq.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "qaz51133"
)

In [ ]:
# 데이터베이스 생성
my_db_name = 'Project_ML'  # create하려는 데이터베이스 이름
sql_query = 'CREATE DATABASE ' + my_db_name

my_cursor = aws_DB.cursor()  # 커서 생성

my_cursor.execute(f"SHOW DATABASES LIKE '{my_db_name}'")  # create하려는 데이터베이스가 이미 있는지 파악을 하기 위함.

# 없으면 생성한다.
if my_cursor.fetchone() is None:
    my_cursor.execute(sql_query)
    aws_DB.commit()
    print(f"데이터베이스 '{my_db_name}'를 생성했습니다.")

# 있으면 스킵한다.
else:  
    print(f"데이터베이스 '{my_db_name}'가 이미 존재합니다. 생성을 스킵합니다.")

my_cursor.execute(f"USE {my_db_name}")  # 해당 데이터베이스로 들어간다.

In [ ]:
# 테이블 생성
my_table_name  = "Table_" +  datetime.datetime.now().strftime('%y%m%d')
sql_query = f'''CREATE TABLE {my_table_name} (
                Class VARCHAR(255) NOT NULL,
                Detected_time TIME,
                Disappeared_time TIME
            )'''

my_cursor.execute(f"SHOW TABLES LIKE %s", (my_table_name,))  # 만드려는 테이블명이 중복인지 아닌지 확인

if my_cursor.fetchone() is None:
    my_cursor.execute(sql_query)
    aws_DB.commit()
    print(f"테이블 '{my_table_name}'를 생성했습니다.")
else:
    print(f"테이블 '{my_table_name}'가 이미 존재합니다. 생성을 스킵합니다.")

In [ ]:
# INSERT 쿼리 불러오기
sql_query = f'''INSERT INTO {my_table_name} (Class, Detected_time) VALUES (%s, %s)'''
sql_query2 = f'''UPDATE {my_table_name} SET Disappeared_time = %s WHERE Class = %s AND Detected_time = %s'''

In [ ]:
# GUI 클래스에 추가할 파라미터 및 Qt 테이블 관련 메소드
class WindowClass(QMainWindow, from_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.setWindowTitle("유해조수 판별 모델 및 추적모델")

        # 객체 검출 정보
        self.record = Camera(self)
        self.previous_status = None
        self.current_status = None
        self.info_table.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        self.row = self.info_table.rowCount()
        
        self.detect_class = None
        self.start_timing = None
        self.end_timing = None

In [ ]:
# 객체 검출 시작/종료 시에 GUI에 표시하거나 
def start_db_record(self):
    self.recordingStart()
    self.row = self.info_table.rowCount()
    self.info_table.insertRow(self.row)
    self.info_table.setItem(self.row, 0, QTableWidgetItem(class_list[self.detect_class]))
    self.start_timing = datetime.datetime.now().strftime('%H:%M:%S')
    self.info_table.setItem(self.row, 1, QTableWidgetItem(str(self.start_timing)))

    my_cursor.execute(sql_query, (class_list[self.detect_class], self.start_timing))
    aws_DB.commit()

def end_db_record(self):
    self.end_timing = datetime.datetime.now().strftime('%H:%M:%S')
    self.info_table.setItem(self.row, 2, QTableWidgetItem(str(self.end_timing)))
    
    my_cursor.execute(sql_query2, (self.end_timing, class_list[self.detect_class], self.start_timing))
    aws_DB.commit()

    self.recordingStop()

In [ ]:
# gui를 닫는 즉시 실행되는 함수
def closeEvent(self, event):
    aws_DB.close()